In [1]:
pip install --upgrade "ipython-sql==0.5.0" "sqlalchemy==2.0.32" "duckdb==1.1.3" "duckdb-engine==0.13.6"

Note: you may need to restart the kernel to use updated packages.


In [31]:
import duckdb
from IPython.core.magic import register_cell_magic

# single shared connection (file-backed so you keep results between cells)
con = duckdb.connect("warehouse/plt.duckdb")


@register_cell_magic
def ducksql(line, cell):
    """Run DuckDB SQL in a cell with %%ducksql"""
    res = con.sql(cell)
    try:
        display(res.df())  # pretty display if it's a query
    except Exception:
        print("OK")  # for DDL/DML statements

In [32]:
%%ducksql
CREATE OR REPLACE TABLE raw_pl AS
SELECT * FROM 'dbt/seeds/premier-league.csv';

OK


In [33]:

%%ducksql
SELECT
    SUM(CASE WHEN club_name IS NULL THEN 1 ELSE 0 END)          AS nulls_club_name,
    SUM(CASE WHEN player_name IS NULL THEN 1 ELSE 0 END)        AS nulls_player_name,
    SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END)                AS nulls_age,
    SUM(CASE WHEN position IS NULL THEN 1 ELSE 0 END)           AS nulls_position,
    SUM(CASE WHEN club_involved_name IS NULL THEN 1 ELSE 0 END) AS nulls_club_involved_name,
    SUM(CASE WHEN fee IS NULL THEN 1 ELSE 0 END)                AS nulls_fee,
    SUM(CASE WHEN transfer_movement IS NULL THEN 1 ELSE 0 END)  AS nulls_transfer_movement,
    SUM(CASE WHEN transfer_period IS NULL THEN 1 ELSE 0 END)    AS nulls_transfer_period,
    SUM(CASE WHEN fee_cleaned IS NULL THEN 1 ELSE 0 END)        AS nulls_fee_cleaned,
    SUM(CASE WHEN league_name IS NULL THEN 1 ELSE 0 END)        AS nulls_league_name,
    SUM(CASE WHEN year IS NULL THEN 1 ELSE 0 END)               AS nulls_year,
    SUM(CASE WHEN season IS NULL THEN 1 ELSE 0 END)             AS nulls_season
FROM raw_pl;

,nulls_club_name,nulls_player_name,nulls_age,nulls_position,nulls_club_involved_name,nulls_fee,nulls_transfer_movement,nulls_transfer_period,nulls_fee_cleaned,nulls_league_name,nulls_year,nulls_season
0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
%%ducksql
SELECT fee, hex(fee)
FROM raw_pl
LIMIT 20;

,fee,hex(fee)
0,€910Th.,E282AC39313054682E
1,€250Th.,E282AC32353054682E
2,?,3F
3,-,2D
4,-,2D
5,€2.00m,E282AC322E30306D
6,loan transfer,6C6F616E207472616E73666572
7,?,3F
8,?,3F
9,?,3F
